In [1]:
image_path = "test_Im.png"
grimace_2_path = "grimace_2.png"
grimace_3_path = "grimace_3.png"
not_a_face_path = "not_a_face.png"

[1, 2, 3, 4, 5, 9, 10, 11, 12, 21, 22,23, 24, 25, 26, 27, 33, 39, 42, 43, 44, 45, 46, 47, 50, 51]

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2

def detect(image_path, min_hand_detection_confidence=0.5, min_hand_presence_confidence=0.5, min_face_detection_confidence=0.5, min_face_presence_confidence=0.5, num_hands=2, dominand_hand='Right', visualize=False, output_face_blendshapes=True, adaptive_threshold=True, max_attempts=3, threshold_reduction_factor=0.7, min_threshold=0.2):
    """
    Detects hands and face in an image, extracts hand landmark coordinates and face blendshapes.
    
    Args:
        image_path (str): Path to the image file
        min_hand_detection_confidence (float): Confidence threshold for hand detection (0.0-1.0)
        min_hand_presence_confidence (float): Confidence threshold for hand presence (0.0-1.0)
        num_hands (int): Maximum number of hands to detect
        dominand_hand (str): Dominant hand preference ('Left' or 'Right')
        visualize (bool): Whether to visualize the results
        output_face_blendshapes (bool): Whether to detect and extract face blendshapes
        
    Returns:
        tuple: (dom_landmarks, non_dom_landmarks, wrists, confidence_scores, detection_status, 
                blendshape_scores, face_landmark_5, face_detected)
               - dom_landmarks: NumPy array of shape [20, 3] with coordinates of dominant hand landmarks
               - non_dom_landmarks: NumPy array of shape [20, 3] with coordinates of non-dominant hand landmarks
               - wrists: NumPy array of shape [2, 2] with coordinates of both wrists [x, y]
               - confidence_scores: NumPy array of shape [2] with confidence scores [dominant_hand, non_dominant_hand]
               - detection_status: NumPy array of shape [2] with binary detection status [dominant_hand, non_dominant_hand]
               - blendshape_scores: NumPy array of shape [26] with selected face blendshape scores
               - face_landmark_5: NumPy array of shape [2] with coordinates of the 5th face landmark [x, y]
               - face_detected: Binary value (1 if face detected, 0 if not)
    """
    # Initialize output arrays for face detection
    blendshape_scores = np.zeros(26)
    nose_landmark = np.zeros(2)
    left_eye_landmark = np.zeros(2)
    right_eye_landmark = np.zeros(2)
    face_detected = 0
    
    # PART 1: HAND LANDMARK DETECTION
    # 1.1: Configure the hand landmarker
    hand_base_options = python.BaseOptions(
        model_asset_path='hand_landmarker.task'
    )

    VisionRunningMode = mp.tasks.vision.RunningMode
    # Configure detection options
    hand_options = vision.HandLandmarkerOptions(
        base_options=hand_base_options,
        num_hands=num_hands,                             
        min_hand_detection_confidence=min_hand_detection_confidence,       
        min_hand_presence_confidence=min_hand_presence_confidence,        
        min_tracking_confidence=0.5,             
        running_mode=VisionRunningMode.IMAGE
    )

    # Create the hand detector
    hand_detector = vision.HandLandmarker.create_from_options(hand_options)

    # 1.2: Load the input image
    image = mp.Image.create_from_file(image_path)

    # 1.3: Detect hand landmarks
    hand_detection_result = hand_detector.detect(image)
    
    # Initialize hand output arrays with zeros
    dom_landmarks = np.zeros((20, 3))       # 20 landmarks (excluding wrist), [x,y,z]
    non_dom_landmarks = np.zeros((20, 3))   # 20 landmarks (excluding wrist), [x,y,z]
    wrists = np.zeros((2, 2))               # 2 wrists, [x,y]
    confidence_scores = np.zeros(2)         # Confidence scores for [dominant, non-dominant]
    detection_status = np.zeros(2, dtype=np.int32)  # Binary detection status [dominant, non-dominant]
    
    # 1.4: Process hand landmarks if hands are detected
    if hand_detection_result.hand_landmarks and hand_detection_result.handedness:
        dom_hand_found = False
        non_dom_hand_found = False
        
        # First, find the dominant and non-dominant hands in detection results
        for idx, handedness in enumerate(hand_detection_result.handedness):
            hand_type = handedness[0].category_name  # 'Left' or 'Right'
            hand_score = handedness[0].score  # Confidence score for the handedness classification
            
            if hand_type == dominand_hand:
                # This is the dominant hand
                dom_hand_found = True
                detection_status[0] = 1  # Set detection status to 1 (detected)
                confidence_scores[0] = hand_score  # Store confidence score
                
                # Store dominant hand wrist coordinates [x,y]
                dom_hand_landmarks = hand_detection_result.hand_landmarks[idx]
                wrists[0, 0] = dom_hand_landmarks[0].x
                wrists[0, 1] = dom_hand_landmarks[0].y
                
                # Store all other dominant hand landmarks (excluding wrist)
                for i in range(1, 21):  # Landmarks 1-20 (skipping wrist which is index 0)
                    dom_landmarks[i-1, 0] = dom_hand_landmarks[i].x
                    dom_landmarks[i-1, 1] = dom_hand_landmarks[i].y
                    dom_landmarks[i-1, 2] = dom_hand_landmarks[i].z
                    
            elif hand_type != dominand_hand:
                # This is the non-dominant hand
                non_dom_hand_found = True
                detection_status[1] = 1  # Set detection status to 1 (detected)
                confidence_scores[1] = hand_score  # Store confidence score
                
                # Store non-dominant hand wrist coordinates [x,y]
                non_dom_hand_landmarks = hand_detection_result.hand_landmarks[idx]
                wrists[1, 0] = non_dom_hand_landmarks[0].x
                wrists[1, 1] = non_dom_hand_landmarks[0].y
                
                # Store all other non-dominant hand landmarks (excluding wrist)
                for i in range(1, 21):  # Landmarks 1-20 (skipping wrist)
                    non_dom_landmarks[i-1, 0] = non_dom_hand_landmarks[i].x
                    non_dom_landmarks[i-1, 1] = non_dom_hand_landmarks[i].y
                    non_dom_landmarks[i-1, 2] = non_dom_hand_landmarks[i].z
                    
        # Log information about which hands were found
        print(f"Dominant hand ({dominand_hand}) detected: {dom_hand_found}")
        print(f"Non-dominant hand detected: {non_dom_hand_found}")
    
    # PART 2: FACE LANDMARK DETECTION (If requested)
    if output_face_blendshapes:
        try:
            # 2.1: Configure the face landmarker
            face_base_options = python.BaseOptions(
                model_asset_path='face_landmarker.task'
            )
            
            # Configure face detection options
            face_options = vision.FaceLandmarkerOptions(
                base_options=face_base_options,
                min_face_detection_confidence = min_face_detection_confidence,
                min_face_presence_confidence = min_face_presence_confidence,
                output_face_blendshapes=True,
                num_faces=1,
                running_mode=VisionRunningMode.IMAGE
            )
            
            # Create the face detector
            face_detector = vision.FaceLandmarker.create_from_options(face_options)
            
            # 2.2: Detect face landmarks (reuse the same image)
            face_detection_result = face_detector.detect(image)
            
            # 2.3: Process face blendshapes if face is detected
            if (face_detection_result.face_blendshapes and len(face_detection_result.face_blendshapes) > 0 and
                face_detection_result.face_landmarks and len(face_detection_result.face_landmarks) > 0):
                
                # Set face detected flag to 1
                face_detected = 1
                
                # Extract the specified blendshape scores (indices [1, 2, 3, 4, 5, 9, 10, 11, 12, 21, 22, 23, 24, 25, 26, 27, 33, 39, 42, 43, 44, 45, 46, 47, 50, 51])
                blendshape_indices = [1, 2, 3, 4, 5, 9, 10, 11, 12, 21, 22, 23, 24, 25, 26, 27, 33, 39, 42, 43, 44, 45, 46, 47, 50, 51]
                
                # Get all blendshapes from the first face
                all_blendshapes = face_detection_result.face_blendshapes[0]
                
                # Fill the blendshape_scores array with the selected scores
                for i, idx in enumerate(blendshape_indices):
                    if idx < len(all_blendshapes):
                        blendshape_scores[i] = all_blendshapes[idx].score
                
 
                #Get nose coordinates
                nose = face_detection_result.face_landmarks[0][4]
                nose_landmark[0] = nose.x
                nose_landmark[1] = nose.y

                #Get eye coordinates
                left_eye = face_detection_result.face_landmarks[0][473]
                left_eye_landmark[0] = left_eye.x
                left_eye_landmark[1] = left_eye.y

                right_eye = face_detection_result.face_landmarks[0][468]
                right_eye_landmark[0] = right_eye.x
                right_eye_landmark[1] = right_eye.y



               
            
        except Exception as e:
            print(f"Error during face detection: {e}")
            # Keep default zero values for face outputs if detection fails
    
    
    
    
    # PART 3: VISUALIZATION
    if visualize:
        # Load the image with OpenCV for visualization
        img_cv = cv2.imread(image_path)
        img_height, img_width, _ = img_cv.shape

        # 3.1: Draw hand landmarks if hands are detected
        if hand_detection_result.hand_landmarks:
            print(f"Visualizing {len(hand_detection_result.hand_landmarks)} hands")
            
            # Define connections between landmarks for hand skeleton
            connections = [
                # Thumb connections
                (0, 1), (1, 2), (2, 3), (3, 4),
                # Index finger connections
                (0, 5), (5, 6), (6, 7), (7, 8),
                # Middle finger connections
                (0, 9), (9, 10), (10, 11), (11, 12),
                # Ring finger connections
                (0, 13), (13, 14), (14, 15), (15, 16),
                # Pinky finger connections
                (0, 17), (17, 18), (18, 19), (19, 20),
                # Palm connections
                (0, 5), (5, 9), (9, 13), (13, 17)
            ]
            
            for idx, hand_landmarks in enumerate(hand_detection_result.hand_landmarks):
                # Determine if this is the dominant hand
                is_dominant = False
                if hand_detection_result.handedness:
                    hand_type = hand_detection_result.handedness[idx][0].category_name
                    is_dominant = (hand_type == dominand_hand)
                
                # Use different colors for dominant vs non-dominant hand
                hand_color = (0, 0, 255) if is_dominant else (255, 0, 0)  # Blue for dominant, Red for non-dominant
                
                # Draw all landmark points
                for landmark in hand_landmarks:
                    # Convert normalized coordinates to pixel coordinates
                    x = int(landmark.x * img_width)
                    y = int(landmark.y * img_height)
                    
                    # Draw the landmark point
                    cv2.circle(img_cv, (x, y), 5, hand_color, -1)
                
                # Draw connections between landmarks (hand skeleton)
                for connection in connections:
                    start_idx, end_idx = connection
                    
                    if start_idx < len(hand_landmarks) and end_idx < len(hand_landmarks):
                        start_point = hand_landmarks[start_idx]
                        end_point = hand_landmarks[end_idx]
                        
                        # Convert normalized coordinates to pixel coordinates
                        start_x = int(start_point.x * img_width)
                        start_y = int(start_point.y * img_height)
                        end_x = int(end_point.x * img_width)
                        end_y = int(end_point.y * img_height)
                        
                        # Draw the connection line
                        cv2.line(img_cv, (start_x, start_y), (end_x, end_y), hand_color, 2)
                
                # Add hand type label (Left/Right, Dominant/Non-dominant)
                if hand_detection_result.handedness:
                    handedness = hand_detection_result.handedness[idx]
                    hand_type = handedness[0].category_name  # 'Left' or 'Right'
                    hand_score = handedness[0].score
                    dom_status = "Dominant" if hand_type == dominand_hand else "Non-dominant"
                    cv2.putText(img_cv, f"{hand_type} Hand - {dom_status} ({hand_score:.2f})", 
                            (10, 30 + idx * 30), cv2.FONT_HERSHEY_SIMPLEX, 
                            0.8, hand_color, 2)
                    
                    # Calculate and draw a bounding box
                    x_coords = [landmark.x for landmark in hand_landmarks]
                    y_coords = [landmark.y for landmark in hand_landmarks]
                    min_x, max_x = min(x_coords), max(x_coords)
                    min_y, max_y = min(y_coords), max(y_coords)
                    
                    # Convert to pixel coordinates
                    min_x, max_x = int(min_x * img_width), int(max_x * img_width)
                    min_y, max_y = int(min_y * img_height), int(max_y * img_height)
                    
                    # Draw bounding box
                    cv2.rectangle(img_cv, (min_x, min_y), (max_x, max_y), hand_color, 2)

        # 3.2: Draw Nose if face was detected
        if face_detected == 1:
            # Convert normalized coordinates to pixel coordinates
            face_x = int(nose_landmark[0] * img_width)
            face_y = int(nose_landmark[1] * img_height)
            
            # Draw the Nose with a distinctive color and size
            cv2.circle(img_cv, (face_x, face_y), 8, (0, 255, 255), -1)  # Yellow circle
            cv2.putText(img_cv, "Nose", (face_x + 10, face_y), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

            # Draw eyes
            left_eye_x = int(left_eye_landmark[0] * img_width)
            left_eye_y = int(left_eye_landmark[1] * img_height)
            right_eye_x = int(right_eye_landmark[0] * img_width)
            right_eye_y = int(right_eye_landmark[1] * img_height)
            
            cv2.circle(img_cv, (left_eye_x, left_eye_y), 6, (255, 255, 0), -1)  # Cyan circle
            cv2.circle(img_cv, (right_eye_x, right_eye_y), 6, (255, 255, 0), -1)  # Cyan circle
            cv2.line(img_cv, (left_eye_x, left_eye_y), (right_eye_x, right_eye_y), (255, 255, 0), 2)
        # 3.3: Add detection status information to visualization
        y_pos = img_height - 80
        hand_status_text = f"Hand Detection: Dom={detection_status[0]}, Non-Dom={detection_status[1]}"
        hand_conf_text = f"Hand Confidence: Dom={confidence_scores[0]:.2f}, Non-Dom={confidence_scores[1]:.2f}"
        face_status_text = f"Face Detection: {face_detected}"
        
        cv2.putText(img_cv, hand_status_text, (10, y_pos), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(img_cv, hand_conf_text, (10, y_pos + 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(img_cv, face_status_text, (10, y_pos + 60), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # 3.4: Display the result
        cv2.imshow('Hand and Face Landmarks', img_cv)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


    if face_detected==1:
        #Calculate distance between the eyes
        eyes_diff = right_eye_landmark-left_eye_landmark
        eyes_distance = np.sqrt(eyes_diff.dot(eyes_diff))
        if detection_status[0]==1 and detection_status[1]==1:
            nose_to_wrist_dist = (wrists-nose_landmark) / eyes_distance
            #Make every hand's landmark potision relative to the wrist, and scaled by the eye's distance
            dom_landmarks[:, 0:2] = (dom_landmarks[:, 0:2] - wrists[0, :]) / eyes_distance
            non_dom_landmarks[:, 0:2] = (non_dom_landmarks[:, 0:2] - wrists[1, :]) / eyes_distance
        elif detection_status[0]==1 and detection_status[1]==0:
            nose_to_wrist_dist = (wrists-nose_landmark) / eyes_distance
            #Make every hand's landmark potision relative to the wrist, and scaled by the eye's distance
            dom_landmarks[:, 0:2] = (dom_landmarks[:, 0:2] - wrists[0, :]) / eyes_distance
        elif detection_status[0]==0 and detection_status[1]==1:
            nose_to_wrist_dist = (wrists-nose_landmark) / eyes_distance
            #Make every hand's landmark potision relative to the wrist, and scaled by the eye's distance
            non_dom_landmarks[:, 0:2] = (non_dom_landmarks[:, 0:2] - wrists[0, :]) / eyes_distance
        
    elif face_detected==0 and detection_status[0]==1:
        #Calculate palm width distance as fallback scaling factor
        palm_width_diff = dom_landmarks[5, :]- dom_landmarks[17, :]
        palm_width_dist = np.sqrt(palm_width_diff.dot(palm_width_diff))
        if detection_status[1]==1:
            nose_to_wrist_dist = (wrists-nose_landmark) / palm_width_dist
            #Make every hand's landmark potision relative to the wrist, and scaled by the palm width 
            dom_landmarks[:, 0:2] = (dom_landmarks[:, 0:2] - wrists[0, :]) / palm_width_dist
            non_dom_landmarks[:, 0:2] = (non_dom_landmarks[:, 0:2] - wrists[1, :]) / palm_width_dist
        elif detection_status[1]==0:
            nose_to_wrist_dist = (wrists-nose_landmark) / palm_width_dist
            #Make every hand's landmark potision relative to the wrist, and scaled by the palm width 
            dom_landmarks[:, 0:2] = (dom_landmarks[:, 0:2] - wrists[0, :]) / palm_width_dist
    elif face_detected==0 and detection_status[0]==0 and detection_status[1]==1:
        #Calculate palm width distance as fallback scaling factor
        palm_width_diff = non_dom_landmarks[5, :]- non_dom_landmarks[17, :]
        palm_width_dist = np.sqrt(palm_width_diff.dot(palm_width_diff))
        nose_to_wrist_dist = (wrists-nose_landmark) / palm_width_dist
        #Make every hand's landmark potision relative to the wrist, and scaled by the palm width 
        non_dom_landmarks[:, 0:2] = (non_dom_landmarks[:, 0:2] - wrists[1, :]) / palm_width_dist
        
    else:
        nose_to_wrist_dist = np.zeros((2, 2))

    
    # Return all requested outputs
    return dom_landmarks, non_dom_landmarks, confidence_scores, detection_status, blendshape_scores, face_detected, nose_to_wrist_dist

In [69]:
lol = detect(grimace_2_path, min_hand_detection_confidence=0.5, min_hand_presence_confidence=0.5, num_hands=2, dominand_hand='Left', visualize=True)

I0000 00:00:1742560531.128181  141863 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1742560531.139864  279116 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.3~git2407250600.76ae27~oibaf~j (git-76ae27e 2024-07-25 jammy-oibaf-ppa)), renderer: AMD Radeon Graphics (radeonsi, renoir, LLVM 15.0.7, DRM 3.42, 5.15.0-131-generic)
W0000 00:00:1742560531.230129  279124 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742560531.257231  279119 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1742560531.387427  141863 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1742560531.390597  279132 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.3~git2407250600.76ae27~oibaf~j (git-76ae27e 2024-07-25 jammy

Dominant hand (Left) detected: True
Non-dominant hand detected: True
Visualizing 2 hands


lol[-1]

In [59]:
lol

(array([[-0.22157899, -0.47131054, -0.03073101],
        [-0.17301513, -0.94633667, -0.04778747],
        [ 0.09016798, -1.17645313, -0.06066544],
        [ 0.44656346, -1.2285489 , -0.07052706],
        [ 0.75239831, -1.00154878, -0.03440262],
        [ 0.74546371, -0.32252427, -0.05854694],
        [ 0.52926255,  0.01389155, -0.07399104],
        [ 0.3284734 ,  0.18489803, -0.08165359],
        [ 0.96142811, -0.83551715, -0.02785412],
        [ 0.733497  , -0.12437514, -0.05066952],
        [ 0.38909197,  0.19143662, -0.05764463],
        [ 0.1054417 ,  0.3583336 , -0.05927849],
        [ 1.04701376, -0.64099421, -0.02452194],
        [ 0.80774103, -0.04213003, -0.04881671],
        [ 0.46820677,  0.20524576, -0.04839684],
        [ 0.19587368,  0.31181122, -0.04229698],
        [ 1.07308434, -0.42637636, -0.02360797],
        [ 1.01975656, -0.01941465, -0.04110365],
        [ 0.78549988,  0.19136538, -0.03911975],
        [ 0.56846131,  0.24917491, -0.03242829]]),
 array([[-0.973760

In [42]:
s = 2

In [40]:
lol[0]

array([[-0.01891005, -0.0402227 , -0.03073101],
       [-0.0147655 , -0.08076251, -0.04778747],
       [ 0.00769514, -0.10040116, -0.06066544],
       [ 0.03811073, -0.10484713, -0.07052706],
       [ 0.06421137, -0.08547443, -0.03440262],
       [ 0.06361955, -0.02752495, -0.05854694],
       [ 0.04516846,  0.00118554, -0.07399104],
       [ 0.02803266,  0.01577961, -0.08165359],
       [ 0.08205044, -0.07130492, -0.02785412],
       [ 0.06259829, -0.01061445, -0.05066952],
       [ 0.03320599,  0.01633763, -0.05764463],
       [ 0.00899863,  0.030581  , -0.05927849],
       [ 0.08935452, -0.05470389, -0.02452194],
       [ 0.06893444, -0.00359547, -0.04881671],
       [ 0.03995782,  0.01751614, -0.04839684],
       [ 0.0167163 ,  0.02661067, -0.04229698],
       [ 0.09157944, -0.03638792, -0.02360797],
       [ 0.08702832, -0.00165689, -0.04110365],
       [ 0.06703633,  0.01633155, -0.03911975],
       [ 0.04851377,  0.02126515, -0.03242829]])

In [50]:
lol[0][:, 0:2] - lol[-2][0, :]

array([[-0.34487933, -0.41592729],
       [-0.34073478, -0.45646709],
       [-0.31827414, -0.47610575],
       [-0.28785855, -0.48055172],
       [-0.26175791, -0.46117902],
       [-0.26234972, -0.40322953],
       [-0.28080082, -0.37451905],
       [-0.29793662, -0.35992497],
       [-0.24391884, -0.4470095 ],
       [-0.26337099, -0.38631904],
       [-0.29276329, -0.35936695],
       [-0.31697065, -0.34512359],
       [-0.23661476, -0.43040848],
       [-0.25703484, -0.37930006],
       [-0.28601146, -0.35818845],
       [-0.30925298, -0.34909391],
       [-0.23438984, -0.41209251],
       [-0.23894095, -0.37736148],
       [-0.25893295, -0.35937303],
       [-0.27745551, -0.35443944]])

In [51]:
(lol[0][:, 0:2] - lol[-2][0, :])/s

array([[-0.17243966, -0.20796365],
       [-0.17036739, -0.22823355],
       [-0.15913707, -0.23805287],
       [-0.14392927, -0.24027586],
       [-0.13087896, -0.23058951],
       [-0.13117486, -0.20161477],
       [-0.14040041, -0.18725953],
       [-0.14896831, -0.17996249],
       [-0.12195942, -0.22350475],
       [-0.1316855 , -0.19315952],
       [-0.14638165, -0.17968348],
       [-0.15848532, -0.17256179],
       [-0.11830738, -0.21520424],
       [-0.12851742, -0.18965003],
       [-0.14300573, -0.17909423],
       [-0.15462649, -0.17454696],
       [-0.11719492, -0.20604625],
       [-0.11947048, -0.18868074],
       [-0.12946647, -0.17968652],
       [-0.13872775, -0.17721972]])

In [32]:
lol[1]

array([[-6.30237758e-01, -3.99838686e-02, -1.02788638e-02],
       [-5.98737776e-01, -6.22401237e-02, -2.47013979e-02],
       [-5.84056258e-01, -7.20628500e-02, -3.81653756e-02],
       [-5.80312997e-01, -6.41725063e-02, -5.22902720e-02],
       [-5.93880087e-01, -2.77553797e-02, -3.40005383e-02],
       [-5.55010289e-01,  4.45717573e-03, -4.82927114e-02],
       [-5.30579254e-01,  2.13617682e-02, -5.71367405e-02],
       [-5.12829348e-01,  3.27022076e-02, -6.22696057e-02],
       [-6.15115538e-01,  5.97834587e-04, -3.67095657e-02],
       [-5.74190766e-01,  2.70960927e-02, -4.77344058e-02],
       [-5.47066554e-01,  4.04051542e-02, -5.11094704e-02],
       [-5.27171567e-01,  4.92314696e-02, -5.39879091e-02],
       [-6.35562539e-01,  2.86360383e-02, -3.92242149e-02],
       [-5.92845291e-01,  5.25679588e-02, -4.87699024e-02],
       [-5.67274824e-01,  6.44097328e-02, -4.67954241e-02],
       [-5.48763901e-01,  6.99670315e-02, -4.47955467e-02],
       [-6.54984802e-01,  5.63003421e-02

In [10]:
lol[6]

array([0.43566036, 0.25460249])

In [11]:
lol[2]-lol[6]

array([[ 0.32596928,  0.37570459],
       [-0.34134416,  0.35962588]])